In [1]:
import numpy as np
import torch
import json
import matplotlib.pyplot as plt

In [2]:
codenames_file = '../data/processed_data/codenames_vecs.json'
dictionary_file = '../data/processed_data/dictionary_vecs.json'

In [6]:
class Codenames:
    def __init__(self,
        codenames_file='../data/processed_data/codenames_vecs.json',
        dictionary_file='../data/processed_data/dictionary_vecs.json'
    ):
        self.codenames_words, self.codenames_vecs = self.load_data(codenames_file)
        self.dictionary_words, self.dictionary_vecs = self.load_data(dictionary_file)
        self.initiate_game()
        self.display_board()

    def load_data(self, file):
        with open(file) as f:
            content = json.load(f)
        words = np.array(list(content.keys()))
        vecs = np.array(list(content.values()))
        return words, vecs

    def initiate_game(self):
        self.cosine_sim_mat = np.matmul(self.dictionary_vecs, self.codenames_vecs.T) # shape = (NUM_DICT, NUM_CODENAMES)
        game_words = np.random.choice(range(len(self.codenames_words)), 25)
        self.remove_game_words_from_dict(game_words)
        self.team_a = game_words[:9]
        self.team_b = game_words[9:17]
        self.neutral = game_words[17:24]
        self.black = game_words[24:25]
        self.game_word_ids = np.random.choice(game_words, 25, replace=False)
        return None
    
    def remove_game_words_from_dict(self, game_words):
        lst_to_remove = []
        for idx in game_words:
            match_num = np.where(self.dictionary_words==self.codenames_words[idx])[0]
            if len(match_num) > 0:
                lst_to_remove.append(match_num[0])
        self.cosine_sim_mat = np.delete(self.cosine_sim_mat, lst_to_remove, 0)
        self.dictionary_words = np.delete(self.dictionary_words, lst_to_remove)
        self.dictionary_vecs = np.delete(self.dictionary_vecs, lst_to_remove)
        return None
        
    def display_board(self):
        print(self.codenames_words[self.game_word_ids])
        return None
    
    def get_team_a_words(self):
        return self.codenames_words[self.team_a]
    
    def get_team_b_words(self):
        return self.codenames_words[self.team_b]
    
    def compute_score(self):
        ########
        # TODO #
        ########
        ret = (
            self.cosine_sim_mat[:, self.team_a].sum(axis=1) -
            2 * self.cosine_sim_mat[:, self.team_b].sum(axis=1) - 
            1 * self.cosine_sim_mat[:, self.neutral].sum(axis=1) -
            5 * self.cosine_sim_mat[:, self.black].sum(axis=1)
        )
        return ret
    
    def find_optimal_num_guess(self, idx):
        ########
        # TODO #
        ########
        print(self.cosine_sim_mat[idx, self.team_a])
        print(self.cosine_sim_mat[idx, self.team_b])
        print(self.cosine_sim_mat[idx, self.neutral])
        print(self.cosine_sim_mat[idx, self.black])
        
    def ai_guess(self):
        idx = self.compute_score().argmax()
        self.find_optimal_num_guess(idx)
        return self.dictionary_words[idx]

In [7]:
a = Codenames()

['yard' 'belt' 'whale' 'apple' 'crown' 'tube' 'plate' 'horseshoe' 'model'
 'brush' 'server' 'phoenix' 'death' 'ghost' 'tie' 'iron' 'lemon' 'lap'
 'ice' 'litter' 'cold' 'strike' 'figure' 'death' 'cotton']


In [8]:
a.ai_guess()

[ 0.00453075  0.07130806  0.04536674 -0.0003753  -0.00236256  0.04929662
  0.1308451   0.1308451   0.05094213]
[ 0.0806136   0.00523256 -0.03445225  0.07224716  0.02917234 -0.04990692
 -0.05155596 -0.04317351]
[ 0.02045589  0.04080355 -0.03125464 -0.05219033 -0.01935559  0.06301405
  0.13145726]
[0.00261078]


'nominated'

In [9]:
a.get_team_a_words()

array(['whale', 'server', 'belt', 'cotton', 'yard', 'lap', 'death',
       'death', 'crown'], dtype='<U11')